In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
import os
from keras.utils import to_categorical

In [ ]:
input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

def encoder(input_img):
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    return encoded

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

def decoder(encoded):
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    return decoded

autoencoder = Model(input_img, decoder(encoder(input_img)))
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# display the weights of the first layer 
autoencoder.summary()

In [ ]:
layer_num = 1
sigma = 0.01

def change_weight(layer_num, sig):
    
    # get the weight matrix for the first layer
    weights = autoencoder.layers[layer_num].get_weights()[0]
    # get the weight matrix for the bias 
    second = autoencoder.layers[layer_num].get_weights()[1]
    
    # In this example, there are total 3x3 arrays of 16 weights
    # it looks like: 
    
    # 16  16  16
    # 16  16  16
    # 16  16  16
    
    for i in range(len(weights[0])): # for each row 
        one = weights[i] 
        for s in range(len(one)): # for the each arrays of the each row
            two = one[s][0]
            for x in range(len(two)): # for the every weights in the arrays
                # update the weight values by using Gaussian by setting mu = weight, and sigma
                two[x] = np.random.normal(two[x], sig, 1)[0] 
    
    # combine the updated weights with the bias weights
    updated = [weights,second] 
    
    # update the weights in the autoencoder layer
    autoencoder.layers[1].set_weights(updated) 
    

    

# change weight. The arguments are (ith layer , sigma)
change_weight(1,0.01)

In [ ]:
num_classes = 10
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

train_y_one_hot = to_categorical(y_train)
test_y_one_hot = to_categorical(y_test)

# Display the change for category label using one-hot encoding
print('Original label:', y_train[0])
print('After conversion to one-hot:', train_y_one_hot[0])

In [ ]:
epoch_num = 10
train_loss = []
val_loss = []
acc = []
val_acc = []

def autoencoder_updator(epochs=epoch_num):
    # intial changing weight
    change_weight(1,0.01)
    change_weight(1,0.02)

    # first epoch of fitting autoencoder
    autoencoder.fit(x_train, x_train,
                    epochs=1,
                    batch_size=256,
                    shuffle=True,
                    validation_data=(x_test, x_test))

    # save the weight for the first epoch of the model
    autoencoder.save_weights('weight.h5')

    
    # starting with 2nd epoch, the weights will be saved and loaded using for loop
    epoch_in = 2

    # 
    for i in range(epoch_num-1):
        print('Epoch number: %s'%(epoch_in))
    
        autoencoder.load_weights('weight.h5')
    
        # update the weight
        change_weight(1,0.01)
        change_weight(1,0.02)
    
        # fit the model once
        history=autoencoder.fit(x_train, x_train,
                    epochs=1,
                    batch_size=256,
                    shuffle=True,
                    validation_data=(x_test, x_test))
    
        # save the model weight
        autoencoder.save_weights('weight.h5')
        
        # store cost
        train_loss.append(history.history['loss'][0])
        val_loss.append(history.history['val_loss'][0])
        #acc.append(history['acc'])
        #val_acc.append(history['val_acc'])
    
        epoch_in += 1


In [ ]:
autoencoder_updator()

In [ ]:
plt.plot([i for i in range(0,epoch_num-1)], train_loss, label='train loss')
plt.plot([i for i in range(0,epoch_num-1)], val_loss, label='validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.title('Loss plot with noise')
plt.legend()

In [ ]:
decoded_imgs = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1+ n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    plt.savefig('reconstruction.png')
plt.show()

In [ ]:
# extracting only encoder and pulling out the encoded imgs
encoder = Model(input_img, encoded)
encoded_imgs = encoder.predict(x_test)
import os
os.chdir('/sfs/qumulo/qhome/yl2vq/MSDS/mnistimage')
# encoded representation
n = 10
plt.figure(figsize=(20, 8))
for i in range(n):
    ax = plt.subplot(1, n, i+1)
    #plt.imshow(encoded_imgs[i].reshape(28,28))
    plt.imshow(encoded_imgs[i].reshape(16, 8).T) # 128 
    plt.gray()
    
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.savefig('1.png')
plt.show()